## Tolman–Oppenheimer–Volkoff equation Solver



### TOV equation

The TOV equation are use to describe the state parameters whose evolution equation are given by:


\begin{equation}
\frac{dp}{dr} = -(\epsilon + p)\frac{m\ +\ 4\pi r^3 p}{r(r\ -\ 2m)} \\
\frac{dm}{dr}=4 \pi r^2 \epsilon
\end{equation}

The above equation uses p, $\epsilon$ and $\rho$ in geometric units (Solar Mass). To convert the geometric units to $g/cm^3$ refer to the cell of Units.

###  Piecewise Polytropes

The Generalised Piecewise Polytropes or GPP are the equations which gives the relation of the state parameters of density pressure and energy density.

The formalism for GPP followed in O_Boyle_2020 is:

Pressure and Dnsity:
\begin{equation}
p\ =\ K\rho^\Gamma\
\end{equation}

Energy density and Density:
\begin{equation}
\epsilon\ =\ \frac{K}{\Gamma\ -\ 1}\rho^\Gamma\ +\ (1\ +\ a)\rho\
\end{equation}


Density and $\eta$:
\begin{equation}
\rho\ =\ \big( \frac{\eta\ -\ a}{K(1\ +\ n)}\big)^n
\end{equation}

Pressure and $\eta$:
\begin{equation}
p\ =\ K\big(\frac{\eta\ -\ a}{K(1\ +\ n)}\big)^{1+n}\
\end{equation}
Energy density and $\eta$:
\begin{equation}
\epsilon\ =\ \rho(\eta)\big(1\ +\ \frac{\eta n\ +\ a}{1\ +\ n} \big)\
\end{equation}
where $\eta\ =\ h\ -\ 1$ and h is the enthalpy, $n\ =\ \frac{1}{\Gamma\ -\ 1}$, K is the polytropic constant, and a is  the Integration constant.

In [10]:
import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.integrate import RK45
from scipy.integrate import solve_ivp
from scipy.integrate import odeint


### Equation of State

The parameters equation of state described Table 1 of the paper O_Boyle_2020 is used in the program to plot the Mass Radius relation. Under this section the user has to mention the EOS to be studied. A list of EOS which are available is given in the cell below.

In [11]:
#EOS parameters available:
#APR, BHF, FPS, H4, KDE0V, KDE0V1, MPA1, MS1, MS1b
#QHC19, RS, SK255, SK272, SKI2, SKI3, SKI4, SKI5,
#SKI6, SKMP, SKOP, SLY2, SLY230A, SLY4, SLY9, WFF1

#Enter the parameters
eos_used='SLY4'

#Compare to Exact EOS
#EOS of Sly4 and QHC19 are not available.
cmp=True

### Units

Here the conversion from [$\mathrm{g\ cm^{-3}}$] to [$M_\odot$] is described. The variable **mul_fac** give the conversion value to be multiplied by the units of [$\mathrm{g\ cm^{-3}}$] to obtatined the quatity in [$M_\odot$].

Note: The units of pressure and energy density is also considered in $\mathrm{g/cm^3}$ as it was used in the paper O_Boyle_2020.

In [12]:
#Units to be converted at the end

G=6.6743*10**-11 #In SI 
c=2.9979*10**8 #In SI
km_m=10**3 
sol_mass=1.989*10**30 #In kg, SI


#Converting M_0 to km
km_M0= km_m * c**2 /(G * sol_mass)

#Multiplication factor to convert from M_0 to km
M0_km=1/km_M0
print(M0_km)

#Multiplication factor to convert from cgs to geometric units
mul_fac=(G/c**2)*(10**9*M0_km**2)
print(mul_fac)

1.477087498725341
1.6202569180215728e-18


### Definition of Pressure, Density and Energy Desnity

For all the functions first the peice index j is matched where the value of the given paramter lies for the Generalised Piecewise Polytrope. After the index j is obtained the parameters K, $\Gamma$, $\Lambda$ and a are used for that piece.

Note: Here $\Lambda$ is an integration constant and not the dimesnionless tidal deformability

1. Pressure as a function of $\eta$:

    Here $\eta$ is the variable hence the peice is selected with respect to $\eta$:

    \begin{equation}
    p\ =\ K_j \big[ \frac{\eta\ -\ a_j}{K_j\ (1\ +\ n_j) } \big]^{1\ +\ n_j} 
    \end{equation}
    
2. $\eta$ as a function of Pressure:

    Here pressure (p) is the variable hence the peice is selected with respect to p:
    
    \begin{equation}
    \eta\ =\ K_j(1\ +\ n_j)\ \big[p{K_j}\big]^{\frac{1}{1\ +\ n_j}} \ +\ a_j
    \end{equation}
3. Pressure as a function of density:

    Here desnity ($\rho$) is the variable hence the peice is selected with respect to $\rho$:
    
    \begin{equation}
    p=K_j \rho^{\Gamma_j}
    \end{equation}
    
4. Density as a funtion of $\eta$:

    Here $\eta$ is the variable hence the peice is selected with respect to $\eta$:
    
    \begin{equation}
    \rho\ =\ \big[ \frac{\eta\ -\ a_j}{K_j(1\ +\ n_j)} \big]^n_j
    \end{equation}
    
5. Energy density as a function of pressure:

    Here pressure (p) is the variable hence the peice is selected with respect to p:
    
    \begin{equation}
    \epsilon\ =\ \frac{p}{\Gamma_j\ -\ 1}\ +\ (1\ +\ a_j)\big(\frac{p}{K_j}\big)^{\frac{1}{\Gamma_j}}
    \end{equation}
    
6. Energy density as a function of $\eta$:

    Here $\eta$ is the variable hence the peice is selected with respect to $\eta$:
    
    \begin{equation}
    \epsilon\ =\ \rho(\eta)\big( 1\ +\ \frac{n_j\eta\ +\ a_j}{1\ +\ n_j} \big)
    \end{equation}
    
7. Energy density as a function of density:

    Here desnity ($\rho$) is the variable hence the peice is selected with respect to $\rho$:
    
    \begin{equation}
    \epsilon\ =\ \frac{K_j}{\Gamma_j\ -\ 1}\rho^\Gamma_j\ +\ (1\ +\ a_j)\rho
    \end{equation}
    

In [13]:
#functions of p rho e and eta

def p_eta(eta):
    
    if eta > eta_th[len(eta_th)-1]:
        j=len(eta_th)-1
    else:
        j=np.argmax(eta<eta_th) - 1
    
    pres= K[j]*((eta - a[j])/(K[j]*(1. + n[j])))**(1. + n[j]) 
    #print(j)
    return pres
    

def eta_p(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<=p_th]) - 1
    
    eta = ((pres/K[j])**(1/(n[j]+1)))*K[j]*(n[j]+1) + a[j]
    
    return eta

def p_rho(rho):
    
    if rho > rho_th[len(rho_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax(rho<rho_th) - 1
    
    pres= K[j]*rho**gma[j] 
    
    #print(j)
    return pres
    
def rho_eta(eta):
    
    if eta > eta_th[len(eta_th)-1]:
        j=len(eta_th)-1
    else:
        j=np.argmax(eta<=eta_th) - 1
    #print(j)
    rho=((eta - a[j])/(K[j]*(1. + n[j])))**(n[j])
    #print(j)
    return rho
    
    
def e_pres(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<p_th]) - 1
    
    edes=((pres)/(gma[j]-1)) + (1. + a[j])*((pres)/K[j])**(1./gma[j])
    
    return edes
    
    
def e_eta(eta):
    
    if eta > eta_th[len(eta_th)-1]:
        j=len(eta_th)-1
    else:
        j=np.argmax(eta<=eta_th) - 1
    
    
    eds=rho_eta(eta)*(1. + ((n[j]*eta + a[j])/(1+n[j])))
    #print(j)
    return eds
    
    
def e_rho(rho):
    
    
    if rho > rho_th[len(rho_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<=rho_th]) - 1
        
    
    eds= (K[j]/(gma[j]-1.))*rho**gma[j] + (1+a[j])*rho
    #print(j)
    #print(eds)
    return eds
    
def rho_pres(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<p_th]) - 1
    
    rho = ((pres)/K[j])**(1./gma[j])
    
    return rho

def drho_dp(pres):
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<p_th]) - 1
    
    drho=(1./(K[j]*gma[j]))*((pres)/K[j])**((1./gma[j]) - 1.)
    
    return drho


def de_dp(pres):
    
     
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<p_th]) - 1
    
    de=(1./(gma[j]-1)) + ((1.+a[j])/K[j]**(1./gma[j]))*(1./gma[j])*(pres)**((1./gma[j]) - 1.)
    
    return de

def dp_drho(rho):
    
    if rho > rho_th[len(p_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<rho_th]) - 1
        
    dp=K[j]*gma[j]*rho**(gma[j]-1.)
    
    return dp

def cs2_speed(rho):
    
    if rho > rho_th[len(p_th)-1]:
        j=len(rho_th)-1
    else:
        j=np.argmax([rho<rho_th]) - 1
        
    cs2=(K[j]*gma[j]*rho**(gma[j]-1.))/(((K[j]*gma[j]*rho**(gma[j]-1.))/(gma[j]-1.))+1.+a[j])
    
    return cs2

### Parameters for PWP:

Here the parameters of the EOS are defined.

rho_th: dividing densities

p_th: pressure at the dividing densities

eta_th: $\eta$ at the dividing densities

K: Polytropic constant

gma: Adiabetic index $\Gamma$

a: Integration constant a

In [14]:
#Parameters of cold matter
#Sly4

#Reading the eos
#This EOS is obtained from the Table 3 of O_Boyle_2020, DOI: https://arxiv.org/abs/2008.03342
#eos=ascii.read('EOS_tables.txt',names=['EOS','lgrho_0','lgK','g1','g2','g3'])

#Selecting the EOS to be used from the file EOS_tables.txt. 
#This files contains all the EOS from Table 3 of the paper O_Boyle_2020



#Using the values of dividning densities of Table 3 from the paper O_Boyle_2020 for the crust region and the core.
#The density joining the crust and the core is used according to the EOS table in use.
rho_th=np.array([0.0,0.0,5.011872336272715e+14,1.000000000000000e+15]) #10.0,2*10**8
#eos_para['lgrho_0'][0]
#Converting cgs to geometric units


#Defining K lambda gamma and a as zeros.
K=np.zeros(len(rho_th))
#lam=np.zeros(len(rho_th))
gma=np.zeros(len(rho_th))
a=np.zeros(len(rho_th))




gma=[1.35692395,3.005,2.988,2.851]


K[0]=3.99873692e-8
#The value of K to have continuity at the density where core and crust join.




a[0]=0.0

lg10p1=13.430358594144145 
lg10rho1=14.7
lg10rho0=(lg10p1 - lg10rho1*gma[1] - np.log10(K[0]))/(gma[0]-gma[1])


rho_th[1]=10.**lg10rho0
rho_th[2]=10.**lg10rho1

#Converting cgs to geometric units
rho_th=rho_th*mul_fac
K[0]=K[0]*(mul_fac)**(1.-gma[0])






#Calculating values of K for the pieces in the core given the condition of continuity and diffrentiability at the  dividing densities.
for i in range(len(K)-1):
    K[i+1]= K[i] * rho_th[i+1]**(gma[i] - gma[i+1])


#rho_th=rho_th*mul_fac
#for i in range(len(K)):
    #K[i]=K[i]*(mul_fac)**(1.-gma[i])


#Calculating values of a for the pieces in the core given the condition of continuity and diffrentiability at the  dividing densities.
for i in range(len(a)-1):
    #a[i+1]= a[i] + (gma[i]*((gma[i+1] - gma[i])/((gma[i+1] - 1.)*(gma[i]-1.))))*K[i]*rho_th[i+1]**(gma[i]-1.)
    a[i+1]=a[i] + K[i]*rho_th[i+1]**(gma[i]-1.)*((gma[i+1] - gma[i])/((gma[i]-1.)*(gma[i+1]-1.)))

#Calculaing n from given gamma
n=[1/(gma[i]-1.) for i in range(len(gma))]

#Calculating the values of eta at the dividing densities.
eta_th=np.array([ ((rho_th[i]**(1./n[i]))*(K[i]*(1. + n[i]))) + a[i] for i in range(len(rho_th))])

#Calculating the values of pressure at the dividing densities.
p_th=np.array([K[j] * rho_th[j]**gma[j] for j in range(len(rho_th)) ])


### TOV evolution Equations

In [15]:
#define TOV equation
def dpm_dr(r,y): 
    
    eta=eta_p(y[0])
    dpdr = - (e_pres(y[0]) + y[0])*(y[1] + 4.0*np.pi*r**3*y[0])/(r*(r-2.0*y[1]))
    dmdr = 4. * np.pi * r**2 * e_pres(y[0]) 
    dmbdr= 4. * np.pi * r**2. * (1. - (2.*y[1]/r))**(-0.5) * rho_pres(y[0])
   
    return np.array([dpdr,dmdr,dmbdr])

def dlm_dr(r,y,y_tdl):
    P=y[0]
    M=y[1]
    rho=rho_pres(P)
    F=(1. - (4.*np.pi*r**2)*(e_pres(P) - P))*(1. - (2.*M/r))**-1
    r2_Q=((4*np.pi*r**2)*(5.*e_pres(P) + 9.*P + (e_pres(P) + P)*(de_dp(rho)))*(1. - (2*M/r))**-1) - (6.*(1. - (2*M/r))**-1.) - ((4.*M**2/r**2)*(1. + (4.*np.pi*r**3*P/M))**2*(1.-(2.*M/r))**-2)
    dy_tdl_dr=-(y_tdl**2 + y_tdl*F + r2_Q)/r
    
    return dy_tdl_dr

### The fourth order Runge Kutta Method

In [16]:
def RK4(r,y,dr):
    
    y_temp =y
    
    k1     = dpm_dr(r,y)
    r,y    = adv_eq(r,y,k1,dr/2.)
    
    k2     = dpm_dr(r,y)
    r,y    = adv_eq(r,y,k2,dr/2.)
    
    k3     = dpm_dr(r, y)
    r,y    = adv_eq(r,y,k3,dr)
    
    k4     = dpm_dr(r,y)
    
    y_new  = y_temp +(dr/6.)*(k1 + 2.*k2 + 2.*k3 + k4)
    
    return y_new
    

def RK4_tdl(r,y,y_tdl,dr):
    
    y_temp_tdl =y_tdl
    
    k1_tdl=dlm_dr(r,y,y_tdl)
    r,y_tdl    = adv_eq_tdl(r,y_tdl,k1_tdl,dr/2.)
    
    k2_tdl=dlm_dr(r,y,y_tdl)
    r,y_tdl    = adv_eq_tdl(r,y_tdl,k2_tdl,dr/2.)
    
    k3_tdl=dlm_dr(r,y,y_tdl)
    r,y_tdl    = adv_eq_tdl(r,y_tdl,k3_tdl,dr)
    
    k4_tdl=dlm_dr(r,y,y_tdl)
    
    y_tdl_new=y_temp_tdl+(dr/6.)*(k1_tdl + 2.*k2_tdl + 2.*k3_tdl + k4_tdl)
    
    return y_tdl_new

    
    
def adv_eq(r,y,k,dr):
    
    #print(y_tdl,k_tdl)
    
    y = y + dr*k
    r = r + dr
    
    return r,y

def adv_eq_tdl(r,y_tdl,k_tdl,dr):
    
    #print(y_tdl,k_tdl)
    
    y_tdl=y_tdl+k_tdl*dr
    r = r + dr
    
    
    return r,y_tdl

### Computing mass and radius

The radius and mass are computed for either 50000 iterations or for pressure above $10^{-12} [\mathrm{M_\odot}]$ with the step size dr = 0.0005 [km].

The initial radius is taken as:
$r\ =\ 10^{-12}$

The initial mass is computed as:

\begin{equation}
m\ =\ \frac{4\pi r^3}{3} \epsilon(p)
\end{equation}

In [17]:
def compute_rm(rho_c):
    #central value and maximum number of steps
    pres        = p_rho(rho_c)
    pres_evl   = [pres]
    Nsteps = 50000 
    
    if pres > p_th[len(p_th)-1]:
        j=len(p_th)-1
    else:
        j=np.argmax([pres<=p_th]) - 1
    
    
    #initialize 
    r = 1e-12
    r_evl=[r]
    dr=0.0005
    m = (4./3.)* np.pi * r**3. *e_pres(pres)
    mb=0.
    m_evl=[m]
    i = 0 
    y_tdl=2.
    #print(pres)

    while ((i<Nsteps-2) & (pres > 10.e-13) ): #10.e-20
        #print(pres)
        y=np.array([pres,m,mb])
        y_tdl=RK4_tdl(r,y,y_tdl,dr)
        y=RK4(r,y,dr)
        #print(y)
        pres=y[0]
        pres_evl.append(pres)
        m=y[1]
        mb=y[2]
        m_evl.append(m)
        r=r+dr
        r_evl.append(r)
        i=i+1
        #print(m)
        
   
    
    C=y[1]/r
    y_new=y_tdl
    
    t1= 2. * C * (6. - 3. * y_new + 3. * C * (5.*y_new - 8.))
    t2= 4. * C**3 * (13. - 11. * y_new + C * (3. * y_new - 2.) + 2. * C**2 * (1. + y_new))
    t3= 3. * (1. - 2. * C)**2 * (2. - y_new + 2. * C *(y_new - 1.)) * np.log(1. - 2.* C)
    
    k2 = (8. * C**5 / 5.) * (1. - 2. * C)**2 * (2. + 2. * C * (y_new-1) - y_new) * (t1 + t2 + t3)**-1
    
    tidal=(2./3.)*k2*r**5
    dim_tidal=(2./3.)*k2*C**-5
    
    return(r,y[1],y[2],dim_tidal,k2)

### Main funtion that generates the values for mass and radius for a given central density.

The central density here is changed and the values of mass and radius is computed through which the Mass and Radius relation is obtained.

The in total 300 iterations for the central density starting from:
$\rho\ =\ 5.0e-4$

and is incremented with each iteration with the step size of $d\rho\ =\ 2.5e-5$


In [18]:

#compute M vs R curve 


########################################### High Resolution ###############################
#iterations=600


#rad=[]
#mas=[]
#eta  = 0.03
#deta = 0.005

##############################################################################################


iterations=300



rad=[]
mas=[]
masb=[]
tid=[]
k_2=[]
rho_c=[]
#eta  = 0.03
#deta = 0.01

rho_c_run=5.0e-4
drho_c_run=2.5e-5


for i in range(iterations): #while (mass > mass_old): 

    #mass_old = mass

    (radius,mass,massb,tidal,k2) = compute_rm(rho_c_run)

    #eta= eta + deta
    print(i,rho_c_run/mul_fac,radius,mass,massb,tidal,k2)
    rho_c.append(rho_c_run)
    rad.append(radius)
    mas.append(mass)
    masb.append(massb)
    tid.append(tidal)
    k_2.append(k2)
    
    rho_c_run=rho_c_run+drho_c_run
    if i==0:
        continue
    else:
        if mas[i-1] > mas[i]:
            break

mas=np.array(mas)
rad=np.array(rad)*M0_km
masb=np.array(masb)


0 308593035116016.56 7.5874999999995145 0.22187551415634157 0.22451177318665383 95139.37137219108 0.0030514404584099995
1 324022686871817.4 7.517999999999553 0.24597466054186495 0.24938348640407856 61267.38062298604 0.0034455679169143674
2 339452338627618.2 7.4719999999995785 0.2713656130335771 0.2756878624247072 40780.52583844684 0.0038648605924836055
3 354881990383419.0 7.442999999999595 0.29796087544431465 0.3033487869315548 27913.59103490087 0.004304900158007975
4 370311642139219.8 7.427499999999603 0.32567563207793 0.3322923070904746 19574.75147827875 0.004758833795791874
5 385741293895020.6 7.421999999999606 0.35442653341535985 0.36244530314865264 14019.290243510932 0.005222081807742597
6 401170945650821.4 7.423999999999605 0.38413106417119947 0.3937347529737408 10228.970341918706 0.005690239020076634
7 416600597406622.2 7.431499999999601 0.4147072624603349 0.42608735838073697 7588.523425389394 0.006159909505770118
8 432030249162423.0 7.443499999999594 0.4460736746497909 0.459429

71 1404098309777875.2 7.346999999999648 1.9388459113453853 2.2779127664630154 5.072423834717218 0.00973807399351529
72 1419527961533676.0 7.330999999999657 1.9465339555014638 2.2891178436351938 4.8635687779737555 0.009628090045074417
73 1434957613289476.8 7.314499999999666 1.953881145554162 2.2998632020969048 4.667349287127849 0.009522019167077688
74 1450387265045277.5 7.297999999999675 1.9608973819375304 2.310159498057238 4.4828235620155406 0.009416682838082474
75 1465816916801078.2 7.281499999999684 1.9675923502176396 2.3200172800306404 4.309007821107456 0.009311934214471596
76 1481246568556879.0 7.264999999999693 1.9739755215031147 2.3294469819733097 4.145181902648026 0.00920800557471822
77 1496676220312679.8 7.248499999999702 1.9800561492876017 2.3384589112028293 3.990711131840317 0.009105176416285896
78 1512105872068480.5 7.2319999999997115 1.9858432735413503 2.34706324708085 3.844811176044393 0.009003244377654025
79 1527535523824281.0 7.214999999999721 1.9913457171849132 2.355270

In [19]:
ascii.write([rho_c,mas,rad,masb,tid,k_2],'SLy4_PwP.dat',names=['rho_c','mass','radius','massb','tidal_dim_less','k2'],overwrite=True)